# مقدمة في الاحتمالات والإحصاء
في هذا الدفتر، سنقوم بالتجربة مع بعض المفاهيم التي ناقشناها سابقًا. العديد من المفاهيم من الاحتمالات والإحصاء ممثلة بشكل جيد في المكتبات الكبرى لمعالجة البيانات في بايثون، مثل `numpy` و `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## المتغيرات العشوائية والتوزيعات
لنبدأ برسم عينة من 30 قيمة من توزيع متساوي بين 0 و 9. سنقوم أيضًا بحساب المتوسط والتباين.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

لتقدير عدد القيم المختلفة في العينة بصريًا، يمكننا رسم **الهيستوجرام**:


In [ ]:
plt.hist(sample)
plt.show()

## تحليل البيانات الحقيقية

المتوسط والتباين مهمان جدًا عند تحليل البيانات الواقعية. لنقم بتحميل البيانات الخاصة بلاعبي البيسبول من [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> نحن نستخدم حزمة تسمى [**Pandas**](https://pandas.pydata.org/) هنا لتحليل البيانات. سنتحدث أكثر عن Pandas والعمل مع البيانات في بايثون لاحقًا في هذه الدورة.

دعونا نحسب القيم المتوسطة للعمر والطول والوزن:


In [ ]:
df[['Age','Height','Weight']].mean()

الآن دعونا نركز على الطول، ونحسب الانحراف المعياري والتباين:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

بالإضافة إلى المتوسط، من المنطقي النظر في القيمة الوسيطية والرباعيات. يمكن تمثيلها بصريًا باستخدام **مخطط الصندوق**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

يمكننا أيضًا صنع مخططات الصناديق لعينات فرعية من مجموعتنا البيانية، على سبيل المثال، مجمعة حسب دور اللاعب.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **ملاحظة**: يشير هذا المخطط إلى أنه في المتوسط، أطوال لاعبي القاعدة الأولى أعلى من أطوال لاعبي القاعدة الثانية. لاحقًا سنتعلم كيف يمكننا اختبار هذه الفرضية بطريقة أكثر رسمية، وكيفية إثبات أن بياناتنا ذات دلالة إحصائية لتوضيح ذلك.

العمر، الطول، والوزن جميعها متغيرات عشوائية مستمرة. ما رأيك في توزيعها؟ طريقة جيدة لمعرفة ذلك هي رسم المدرج التكراري للقيم:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## التوزيع الطبيعي

دعونا ننشئ عينة اصطناعية للأوزان تتبع توزيعًا طبيعيًا بمتوسط وتباين متماثل مع بياناتنا الحقيقية:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

نظرًا لأن معظم القيم في الحياة الواقعية تتبع التوزيع الطبيعي، فلا ينبغي علينا استخدام مولد أعداد عشوائية متساوية التوزيع لتوليد بيانات عينة. إليك ما يحدث إذا حاولنا توليد الأوزان بتوزيع متساوي (تم توليدها بواسطة `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## فترات الثقة

دعونا الآن نحسب فترات الثقة لأوزان وأطوال لاعبي البيسبول. سنستخدم الكود [من هذا النقاش على ستاك أوفرفلو](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## اختبار الفرضيات

دعونا نستكشف أدوارًا مختلفة في مجموعة بيانات لاعبي البيسبول لدينا:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

دعونا نختبر الفرضية القائلة بأن لاعبي القاعدة الأولى أطول من لاعبي القاعدة الثانية. أبسط طريقة للقيام بذلك هي اختبار فترات الثقة:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

يمكننا أن نرى أن الفواصل الزمنية لا تتداخل.

الطريقة الأكثر صحة إحصائيًا لإثبات الفرضية هي استخدام **اختبار t لستودنت**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

القيمتان اللتان تعيدهما دالة `ttest_ind` هما:
* قيمة p يمكن اعتبارها كاحتمال أن يكون للوزعين نفس المتوسط. في حالتنا، القيمة منخفضة جدًا، مما يعني أن هناك دليلاً قوياً يدعم أن لاعبي القاعدة الأولى أطول.
* قيمة t هي القيمة الوسيطة لاختلاف المتوسطات المعيارية التي تُستخدم في اختبار t، ويتم مقارنتها مع قيمة عتبة لقيمة ثقة معينة.


## محاكاة توزيع طبيعي مع نظرية الحد المركزي

مصدر الأعداد العشوائية الزائفة في بايثون مصمم لإعطائنا توزيعًا منتظمًا. إذا أردنا إنشاء مولد لتوزيع طبيعي، يمكننا استخدام نظرية الحد المركزي. للحصول على قيمة موزعة طبيعيًا، سنقوم فقط بحساب متوسط عينة مولدة بشكل منتظم.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## الارتباط وشركة البيسبول الشريرة

يسمح لنا الارتباط بإيجاد العلاقات بين تسلسلات البيانات. في مثالنا التوضيحي، دعونا نتخيل وجود شركة بيسبول شريرة تدفع للاعبيها وفقًا لطولهم - كلما كان اللاعب أطول، زادت الأموال التي يحصل عليها. لنفترض أن هناك راتبًا أساسيًا قدره 1000 دولار، ومكافأة إضافية تتراوح بين 0 و100 دولار، تعتمد على الطول. سنأخذ اللاعبين الحقيقيين من MLB، ونحسب رواتبهم الخيالية:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

لنحسب الآن التغاير والارتباط لتلك المتتاليات. ستعطينا الدالة `np.cov` ما يُسمى **مصفوفة التغاير**، وهي تمديد للتغاير ليشمل متغيرات متعددة. العنصر $M_{ij}$ من مصفوفة التغاير $M$ هو التغاير بين المتغيرين الإدخاليين $X_i$ و $X_j$، والقيم القطرية $M_{ii}$ هي تباين $X_{i}$. وبالمثل، ستعطينا الدالة `np.corrcoef` **مصفوفة الارتباط**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

يشير الارتباط الذي يساوي 1 إلى وجود **علاقة خطية** قوية بين متغيرين. يمكننا رؤية العلاقة الخطية بصريًا من خلال رسم قيمة مقابل الأخرى:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

لنرَ ماذا يحدث إذا لم تكن العلاقة خطية. لنفترض أن شركتنا قررت إخفاء التبعية الخطية الواضحة بين الطول والرواتب، وأدخلت بعض اللاخطية في الصيغة، مثل `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

في هذه الحالة، الارتباط أقل قليلاً، ولكنه لا يزال مرتفعًا إلى حد كبير. الآن، لجعل العلاقة أقل وضوحًا، قد نرغب في إضافة بعض العشوائية الإضافية عن طريق إضافة متغير عشوائي إلى الراتب. لنرَ ما سيحدث:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> هل يمكنك تخمين لماذا تتراصف النقاط في خطوط عمودية مثل هذه؟

لقد لاحظنا العلاقة بين مفهوم مصطنع الهندسة مثل الراتب والمتغير الملحوظ *الطول*. دعونا نرى أيضًا ما إذا كان المتغيران الملحظان، مثل الطول والوزن، يرتبطان أيضًا:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

لسوء الحظ، لم نحصل على أي نتائج - فقط بعض القيم الغريبة `nan`. يرجع ذلك إلى أن بعض القيم في سلسلتنا غير معرفة، ممثلة بـ `nan`، مما يسبب أن تكون نتيجة العملية غير معرفة أيضًا. من خلال النظر إلى المصفوفة يمكننا أن نرى أن عمود `Weight` هو العمود الإشكالي، لأن الارتباط الذاتي بين قيم `Height` قد تم حسابه.

> يوضح هذا المثال أهمية **إعداد البيانات** و**تنظيفها**. بدون بيانات مناسبة لا يمكننا حساب أي شيء.

لنعتمد على طريقة `fillna` لملء القيم المفقودة، وحساب الارتباط:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

هناك بالفعل علاقة ارتباط، لكنها ليست قوية كما في مثالنا الاصطناعي. في الواقع، إذا نظرنا إلى مخطط التشتت لقيمة مقابل الأخرى، فإن العلاقة ستكون أقل وضوحًا بكثير:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## الاستنتاج

في هذا الدفتر تعلمنا كيفية إجراء العمليات الأساسية على البيانات لحساب الدوال الإحصائية. نحن الآن نعرف كيفية استخدام مجموعة قوية من الرياضيات والإحصائيات من أجل إثبات بعض الفرضيات، وكيفية حساب فواصل الثقة للمتغيرات التعسفية بناءً على عينة بيانات.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**تنويه**:
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى جاهدين لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. ينبغي اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق به. للمعلومات الحساسة، يُنصح بالاعتماد على الترجمة المهنية بواسطة مترجم بشري. نحن غير مسؤولين عن أي سوء تفاهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
